<a href="https://colab.research.google.com/github/Mengfan-Wei/colab/blob/main/chapter_deep-learning-computation/use-gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU


In [1]:
!nvidia-smi

Fri Sep 16 07:10:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------



## [**计算设备**]

我们可以指定用于存储和计算的设备，如CPU和GPU。
默认情况下，张量是在内存中创建的，然后使用CPU计算它。


在PyTorch中，CPU和GPU可以用`torch.device('cpu')`
和`torch.device('cuda')`表示。
应该注意的是，`cpu`设备意味着所有物理CPU和内存，
这意味着PyTorch的计算将尝试使用所有CPU核心。
然而，`gpu`设备只代表一个卡和相应的显存。
如果有多个GPU，我们使用`torch.device(f'cuda:{i}')`
来表示第$i$块GPU（$i$从0开始）。
另外，`cuda:0`和`cuda`是等价的。


In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [3]:
torch.cuda.device_count()

1

现在我们定义了两个方便的函数，
[**这两个函数允许我们在不存在所需所有GPU的情况下运行代码。**]


In [4]:
def try_gpu(i=0): 
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## 张量与GPU

我们可以[**查询张量所在的设备。**]
默认情况下，张量是在CPU上创建的。


In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

需要注意的是，无论何时我们要对多个项进行操作，
它们都必须在同一个设备上。

### [**存储在GPU上**]

我们在第一个`gpu`上创建张量变量`X`。
在GPU上创建的张量只消耗这个GPU的显存。


In [7]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

假设你至少有两个GPU，下面的代码将在(**第二个GPU上创建一个随机张量。**)


In [ ]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.1379, 0.4532, 0.9869],
        [0.4779, 0.7426, 0.9622]], device='cuda:1')

### 复制

如果我们[**要计算`X + Y`，我们需要决定在哪里执行这个操作**]。

![复制数据以在同一设备上执行操作](http://d2l.ai/_images/copyto.svg)
:label:`fig_copyto`


In [ ]:
Z = X.cuda(1)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')


[**现在数据在同一个GPU上（`Z`和`Y`都在），我们可以将它们相加。**]


In [ ]:
Y + Z

tensor([[1.1379, 1.4532, 1.9869],
        [1.4779, 1.7426, 1.9622]], device='cuda:1')

假设变量`Z`已经存在于第二个GPU上。
如果我们还是调用`Z.cuda(1)`会发生什么？
它将返回`Z`，而不会复制并分配新内存。


In [ ]:
Z.cuda(1) is Z

True

### 旁注

在设备（CPU、GPU和其他机器）之间传输数据比计算慢得多。

## [**神经网络与GPU**]

In [8]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())  # 将神经网络模型的所有参数在0号GPU上copy一份

net(X)  # 输入同在GPU：0上的张量，模型将在同一GPU上计算结果

tensor([[-0.4731],
        [-0.4731]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
net(X)

tensor([[-0.0132],
        [-0.0132]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
net[0].weight.data.device # 确认模型参数存储在同一个GPU上

device(type='cuda', index=0)